In [1]:
import mediapipe
from cvzone.HandTrackingModule import HandDetector
import cv2
import autopy

In [2]:
wScr,hScr = autopy.screen.size()
print(wScr,hScr)

1536.0 864.0


In [5]:
cap = cv2.VideoCapture(0)
cap.set(3,1536)
cap.set(4,864)
detector = HandDetector(detectionCon = 0.8, maxHands=1)
myEquation = ""
delayCounter = 0

#Creating Class for Buttons
class Button:
    def __init__(self,pos,width,height,value):   
        self.pos = pos
        self.width = width
        self.height = height
        self.value = value
        
    def draw(self,img):
        cv2.rectangle(img,self.pos,(self.pos[0]+self.width,self.pos[1]+self.height),(225,225,225),cv2.FILLED)
        cv2.rectangle(img,self.pos,(self.pos[0]+self.width,self.pos[1]+self.height),(25,25,25),4)
        cv2.putText(img,self.value,(self.pos[0]+30,self.pos[1]+70),cv2.FONT_HERSHEY_SIMPLEX,2,(50,50,50),2)
    
    def checkClick(self,x,y):
        if self.pos[0]<x<self.pos[0]+self.width and self.pos[1]<y<self.pos[1]+self.height:
            cv2.rectangle(img,self.pos,(self.pos[0]+self.width,self.pos[1]+self.height),(255,255,255),cv2.FILLED)
            cv2.rectangle(img,self.pos,(self.pos[0]+self.width,self.pos[1]+self.height),(25,25,25),4)
            cv2.putText(img,self.value,(self.pos[0]+20,self.pos[1]+80),cv2.FONT_HERSHEY_SIMPLEX,3,(50,50,50),3)
            return True
        else:
            return False

        
buttonValues = [["7", "8", "9","*" ],
               ["4", "5", "6","-" ],
               ["1", "2", "3","+" ],
               ["0", "/", ".","=" ],]
buttonList = []

for i in range(4):
    for j in range(4):
        xpos = i*100+800
        ypos = j*100+150
        buttonList.append(Button((xpos,ypos),100,100,buttonValues[j][i]))


#Loop
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img= detector.findHands(img, flipType=False)
    
    cv2.rectangle(img,(800,40),(800+400,40+110),(225,225,225),cv2.FILLED)
    cv2.rectangle(img,(800,40),(800+400,40+110),(50,50,50),4)
    cv2.rectangle(img,(1100,65),(1100+80,65+60),(225,225,225),cv2.FILLED)
    cv2.rectangle(img,(1100,65),(1100+80,65+60),(50,50,50),4)
    cv2.putText(img,"X",(1100+20,65+50),cv2.FONT_HERSHEY_SIMPLEX,2,(50,50,50),2)
    

    
    for button in buttonList:
        button.draw(img)
        
         
    if hands:
        lmList = hands[0]['lmList']
        length, _, img= detector.findDistance((lmList[8][0],lmList[8][1]),(lmList[12][0],lmList[12][1]),img)
        x,y = lmList[8][0],lmList[8][1]
        if length<50:
            for i,button in enumerate(buttonList):
                if button.checkClick(x,y) and delayCounter == 0:
                    myValue = buttonValues[int(i%4)][int(i/4)]
                    if myValue == "=":
                        myEquation = str(eval(myEquation))
                    else:
                        myEquation += myValue
                    delayCounter = 1
                    
    if 1100<lmList[8][0]<1100+80 and 65<lmList[8][1]<65+60:
        cv2.rectangle(img,(1100,65),(1100+80,65+60),(0,255,0),cv2.FILLED)
        cv2.rectangle(img,(1100,65),(1100+80,65+60),(50,50,50),4)
        cv2.putText(img,"X",(1100+20,65+50),cv2.FONT_HERSHEY_SIMPLEX,2,(50,50,50),2)
        
        myEquation = ""
        
    
    if delayCounter !=0:
        delayCounter +=1
        if delayCounter >10:
            delayCounter = 0
        
    
    cv2.putText(img,myEquation,(800+30,40+70),cv2.FONT_HERSHEY_SIMPLEX,2,(50,50,50),2)
#     Display
    cv2.imshow("Image",img)
    cv2.waitKey(1)
    

KeyboardInterrupt: 